In [1]:
# BPI2020 Domestic Declarations - Statistical Analysis
# ==================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load and prepare the dataset
print("Loading and preparing the dataset...")
df = pd.read_csv('input/BPI2020_DomesticDeclarations.csv')

# Convert timestamp to datetime
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'])



Loading and preparing the dataset...


In [2]:
# 1. Preprocessing for statistical analysis
# =======================================

# Calculate case durations
print("Calculating case metrics...")
case_start = df.groupby('case:id')['time:timestamp'].min().reset_index()
case_end = df.groupby('case:id')['time:timestamp'].max().reset_index()
case_duration = pd.merge(case_start, case_end, on='case:id', suffixes=('_start', '_end'))
case_duration['duration_days'] = (case_duration['time:timestamp_end'] - case_duration['time:timestamp_start']).dt.total_seconds() / (60*60*24)

# Add month information
case_duration['month'] = case_duration['time:timestamp_start'].dt.month
case_duration['year'] = case_duration['time:timestamp_start'].dt.year
case_duration['year_month'] = case_duration['time:timestamp_start'].dt.strftime('%Y-%m')

# Count activities per case
activity_counts = df.groupby('case:id').size().reset_index(name='activity_count')

# Get the first amount for each case (should be the same for all activities in a case)
case_amounts = df.groupby('case:id')['case:Amount'].first().reset_index()

# Combine all case metrics
case_data = pd.merge(case_duration, activity_counts, on='case:id')
case_data = pd.merge(case_data, case_amounts, on='case:id')

# Add information about roles involved in each case
roles_per_case = df.groupby('case:id')['org:role'].nunique().reset_index(name='unique_roles')
case_data = pd.merge(case_data, roles_per_case, on='case:id')

# Extract process variants
df_sorted = df.sort_values(['case:id', 'time:timestamp'])
case_sequences = df_sorted.groupby('case:id')['concept:name'].apply(list).reset_index()
case_sequences['sequence_str'] = case_sequences['concept:name'].apply(lambda x: ' -> '.join(x))

# Get the most common variants
variant_counts = case_sequences['sequence_str'].value_counts().reset_index()
variant_counts.columns = ['variant', 'count']
top_variants = variant_counts.head(5)['variant'].tolist()

# Mark cases with common variants
case_sequences['is_common_variant'] = case_sequences['sequence_str'].isin(top_variants)
case_sequences['variant_id'] = case_sequences['sequence_str'].map(
    {variant: f"Variant {i+1}" for i, variant in enumerate(top_variants)}
)
case_sequences.loc[~case_sequences['is_common_variant'], 'variant_id'] = 'Other'

# Merge variant information with case data
case_data = pd.merge(case_data, case_sequences[['case:id', 'is_common_variant', 'variant_id']], on='case:id')



Calculating case metrics...


In [3]:
# 2. Descriptive Statistics
# =======================

print("\n--- Descriptive Statistics ---")

# Basic statistics for case duration
duration_stats = case_data['duration_days'].describe()
print("\nCase Duration Statistics (days):")
print(duration_stats)

# Basic statistics for declaration amounts
amount_stats = case_data['case:Amount'].describe()
print("\nDeclaration Amount Statistics:")
print(amount_stats)

# Activity count statistics
activity_stats = case_data['activity_count'].describe()
print("\nActivities per Case Statistics:")
print(activity_stats)




--- Descriptive Statistics ---

Case Duration Statistics (days):
count    10500.000000
mean        11.525551
std         17.020116
min          0.000000
25%          5.817023
50%          7.330509
75%         12.241227
max        469.277986
Name: duration_days, dtype: float64

Declaration Amount Statistics:
count    10500.000000
mean        86.419636
std        144.537177
min          0.000000
25%         22.161465
50%         41.506246
75%         87.331995
max       3292.536991
Name: case:Amount, dtype: float64

Activities per Case Statistics:
count    10500.000000
mean         5.374952
std          1.486415
min          1.000000
25%          5.000000
50%          5.000000
75%          6.000000
max         24.000000
Name: activity_count, dtype: float64


In [4]:
# 3. Hypothesis Testing
# ===================

print("\n--- Hypothesis Testing ---")

# Test 1: Is there a significant difference in case duration between common variants and other cases?
common_duration = case_data[case_data['is_common_variant']]['duration_days']
other_duration = case_data[~case_data['is_common_variant']]['duration_days']

t_stat, p_val = stats.ttest_ind(common_duration, other_duration, equal_var=False)
print("\nT-test: Case Duration - Common Variants vs Others")
print(f"T-statistic: {t_stat:.4f}, p-value: {p_val:.4f}")
print(f"Common variants mean duration: {common_duration.mean():.2f} days")
print(f"Other variants mean duration: {other_duration.mean():.2f} days")
print(f"Conclusion: {'Significant difference' if p_val < 0.05 else 'No significant difference'}")

# Visualize the difference
fig = go.Figure()
fig.add_trace(go.Box(
    y=common_duration,
    name='Common Variants',
    boxmean=True
))
fig.add_trace(go.Box(
    y=other_duration,
    name='Other Variants',
    boxmean=True
))
fig.update_layout(
    title='Case Duration Comparison: Common Variants vs Others',
    yaxis_title='Duration (days)',
    showlegend=True
)
fig.show()




--- Hypothesis Testing ---

T-test: Case Duration - Common Variants vs Others
T-statistic: -7.3617, p-value: 0.0000
Common variants mean duration: 10.66 days
Other variants mean duration: 18.93 days
Conclusion: Significant difference


In [5]:
# Test 2: Is there a significant difference in amounts across top variants?
print("\nANOVA: Declaration Amount Across Top Variants")
variant_groups = [case_data[case_data['variant_id'] == variant]['case:Amount'] for variant in 
                 case_data['variant_id'].unique() if variant != 'Other']

if len(variant_groups) >= 2:
    f_stat, p_val = stats.f_oneway(*variant_groups)
    print(f"F-statistic: {f_stat:.4f}, p-value: {p_val:.4f}")
    print(f"Conclusion: {'Significant difference' if p_val < 0.05 else 'No significant difference'}")

    # Visualize amounts by variant
    fig = px.box(
        case_data[case_data['variant_id'] != 'Other'],
        x='variant_id',
        y='case:Amount',
        title='Declaration Amounts by Process Variant',
        color='variant_id'
    )
    fig.update_layout(showlegend=False)
    fig.show()




ANOVA: Declaration Amount Across Top Variants
F-statistic: 28.9566, p-value: 0.0000
Conclusion: Significant difference


In [6]:
# Test 3: Correlation between case duration and declaration amount
corr, p_val = stats.pearsonr(case_data['duration_days'], case_data['case:Amount'])
print("\nCorrelation: Case Duration vs Declaration Amount")
print(f"Pearson correlation: {corr:.4f}, p-value: {p_val:.4f}")
print(f"Conclusion: {'Significant correlation' if p_val < 0.05 else 'No significant correlation'}")

# Visualize correlation
fig = px.scatter(
    case_data,
    x='duration_days',
    y='case:Amount',
    trendline='ols',
    title=f'Case Duration vs Declaration Amount (r={corr:.4f})',
    labels={'duration_days': 'Case Duration (days)', 'case:Amount': 'Declaration Amount'}
)
fig.show()




Correlation: Case Duration vs Declaration Amount
Pearson correlation: 0.0619, p-value: 0.0000
Conclusion: Significant correlation


In [7]:
# Test 4: Is there seasonality in case durations (by month)?
monthly_durations = case_data.groupby('month')['duration_days'].mean().reset_index()

f_stat, p_val = stats.f_oneway(*[
    case_data[case_data['month'] == month]['duration_days'] 
    for month in case_data['month'].unique()
])
print("\nANOVA: Case Duration by Month")
print(f"F-statistic: {f_stat:.4f}, p-value: {p_val:.4f}")
print(f"Conclusion: {'Significant seasonal effect' if p_val < 0.05 else 'No significant seasonal effect'}")

# Visualize monthly trends
fig = px.line(
    monthly_durations,
    x='month',
    y='duration_days',
    title='Average Case Duration by Month',
    markers=True,
    labels={'month': 'Month', 'duration_days': 'Average Duration (days)'}
)
fig.update_layout(xaxis=dict(tickmode='linear', dtick=1))
fig.show()




ANOVA: Case Duration by Month
F-statistic: 5.0590, p-value: 0.0000
Conclusion: Significant seasonal effect


In [8]:
# 4. Regression Analysis
# ====================

print("\n--- Regression Analysis ---")

# Predicting case duration
print("\nPredicting Case Duration:")
X = sm.add_constant(case_data[['activity_count', 'case:Amount', 'unique_roles']])
y = case_data['duration_days']

model = sm.OLS(y, X).fit()
print(model.summary())

# Visualize regression results
predictions = model.predict(X)
case_data['predicted_duration'] = predictions

fig = px.scatter(
    case_data,
    x='predicted_duration',
    y='duration_days',
    color='activity_count',
    title='Actual vs Predicted Case Duration',
    labels={'predicted_duration': 'Predicted Duration (days)', 
            'duration_days': 'Actual Duration (days)',
            'activity_count': 'Activity Count'}
)
fig.add_trace(
    go.Scatter(
        x=[0, case_data['duration_days'].max()],
        y=[0, case_data['duration_days'].max()],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='Perfect Prediction'
    )
)
fig.show()




--- Regression Analysis ---

Predicting Case Duration:
                            OLS Regression Results                            
Dep. Variable:          duration_days   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     201.4
Date:                Mon, 10 Mar 2025   Prob (F-statistic):          4.87e-127
Time:                        16:59:12   Log-Likelihood:                -44366.
No. Observations:               10500   AIC:                         8.874e+04
Df Residuals:                   10496   BIC:                         8.877e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [9]:
# 5. Time Series Analysis
# =====================

print("\n--- Time Series Analysis ---")

# Analyze trends in process metrics over time
case_data['year_month'] = pd.to_datetime(case_data['year_month'])
monthly_metrics = case_data.groupby('year_month').agg({
    'duration_days': 'mean',
    'activity_count': 'mean',
    'case:Amount': 'mean',
    'case:id': 'count'
}).reset_index()
monthly_metrics.columns = ['year_month', 'avg_duration', 'avg_activities', 'avg_amount', 'case_count']

# Create a combined plot
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=('Average Case Duration', 'Average Activities per Case',
                                   'Average Declaration Amount', 'Number of Cases'),
                    shared_xaxes=True)

# Duration trend
fig.add_trace(
    go.Scatter(x=monthly_metrics['year_month'], y=monthly_metrics['avg_duration'], 
               mode='lines+markers', name='Avg Duration'),
    row=1, col=1
)

# Activity count trend
fig.add_trace(
    go.Scatter(x=monthly_metrics['year_month'], y=monthly_metrics['avg_activities'], 
               mode='lines+markers', name='Avg Activities'),
    row=1, col=2
)

# Amount trend
fig.add_trace(
    go.Scatter(x=monthly_metrics['year_month'], y=monthly_metrics['avg_amount'], 
               mode='lines+markers', name='Avg Amount'),
    row=2, col=1
)

# Case count trend
fig.add_trace(
    go.Scatter(x=monthly_metrics['year_month'], y=monthly_metrics['case_count'], 
               mode='lines+markers', name='Case Count'),
    row=2, col=2
)

fig.update_layout(
    height=800,
    title_text="Process Metrics Over Time",
    showlegend=False
)
fig.show()




--- Time Series Analysis ---


In [10]:
# 6. Performance Benchmarking
# =========================

print("\n--- Performance Benchmarking ---")

# Compare performance metrics across different process variants
variant_performance = case_data.groupby('variant_id').agg({
    'duration_days': ['mean', 'median', 'std', 'count'],
    'activity_count': 'mean',
    'case:Amount': 'mean'
}).reset_index()

# Flatten multi-level columns
variant_performance.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in variant_performance.columns]

print("\nPerformance Metrics by Process Variant:")
print(variant_performance)

# Map variant_id to numerical values for coloring
variant_id_mapping = {variant: idx for idx, variant in enumerate(variant_performance['variant_id'].unique())}
variant_performance['variant_id_num'] = variant_performance['variant_id'].map(variant_id_mapping)

# Visualize performance comparison
fig = px.parallel_coordinates(
    variant_performance,
    dimensions=['duration_days_mean', 'activity_count_mean', 'case:Amount_mean', 'duration_days_count'],
    color='variant_id_num',
    labels={
        'duration_days_mean': 'Avg Duration (days)',
        'activity_count_mean': 'Avg Activities',
        'case:Amount_mean': 'Avg Amount',
        'duration_days_count': 'Case Count',
        'variant_id_num': 'Variant ID'
    },
    title='Process Variant Performance Comparison'
)
fig.show()




--- Performance Benchmarking ---

Performance Metrics by Process Variant:
  variant_id  duration_days_mean  duration_days_median  duration_days_std  \
0      Other           18.934415             10.173391          36.975197   
1  Variant 1            9.377175              7.036163          11.215082   
2  Variant 2           12.550213             10.232593          10.484254   
3  Variant 3            9.891597              6.311244          15.539418   
4  Variant 4            9.965954              7.309352           9.887759   
5  Variant 5           18.571709             11.104410          23.635159   

   duration_days_count  activity_count_mean  case:Amount_mean  
0                 1097             6.659982         96.805316  
1                 4618             5.000000         81.869265  
2                 2473             6.000000         84.739083  
3                 1392             4.000000         81.286216  
4                  575             5.000000         78.232542  
5

In [11]:
# 7. Correlation Matrix
# ===================

print("\n--- Correlation Analysis ---")

# Calculate correlation matrix for case metrics
corr_metrics = ['duration_days', 'activity_count', 'case:Amount', 'unique_roles']
correlation_matrix = case_data[corr_metrics].corr()

print("\nCorrelation Matrix:")
print(correlation_matrix)

# Visualize correlation matrix
fig = px.imshow(
    correlation_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    title='Correlation Matrix of Case Metrics',
    zmin=-1,
    zmax=1
)
fig.show()




--- Correlation Analysis ---

Correlation Matrix:
                duration_days  activity_count  case:Amount  unique_roles
duration_days        1.000000        0.227462     0.061945      0.108239
activity_count       0.227462        1.000000     0.187540      0.640678
case:Amount          0.061945        0.187540     1.000000      0.093410
unique_roles         0.108239        0.640678     0.093410      1.000000


In [12]:
# 8. Process Efficiency Metrics
# ===========================

print("\n--- Process Efficiency Metrics ---")

# Calculate efficiency metrics
total_cases = len(case_data)
avg_duration = case_data['duration_days'].mean()
avg_activities = case_data['activity_count'].mean()
efficiency_ratio = 100 / (avg_duration * avg_activities)  # Higher is better

print(f"\nProcess Efficiency Metrics:")
print(f"Total cases processed: {total_cases}")
print(f"Average case duration: {avg_duration:.2f} days")
print(f"Average activities per case: {avg_activities:.2f}")
print(f"Efficiency ratio: {efficiency_ratio:.4f}")

# Calculate efficiency by variant
variant_efficiency = variant_performance.copy()
variant_efficiency['efficiency_ratio'] = 100 / (variant_efficiency['duration_days_mean'] * variant_efficiency['activity_count_mean'])

# Visualize efficiency by variant
fig = px.bar(
    variant_efficiency,
    x='variant_id',
    y='efficiency_ratio',
    color='variant_id',
    text='efficiency_ratio',
    title='Process Efficiency by Variant',
    labels={'efficiency_ratio': 'Efficiency Ratio', 'variant_id': 'Process Variant'}
)
fig.update_traces(texttemplate='%{text:.4f}', textposition='outside')
fig.show()




--- Process Efficiency Metrics ---

Process Efficiency Metrics:
Total cases processed: 10500
Average case duration: 11.53 days
Average activities per case: 5.37
Efficiency ratio: 1.6142


In [13]:
# 9. Advanced Statistical Tests
# ===========================

print("\n--- Advanced Statistical Tests ---")

# Chi-square test for association between process variant and month
contingency_table = pd.crosstab(case_data['variant_id'], case_data['month'])
chi2, p_val, dof, expected = stats.chi2_contingency(contingency_table)

print("\nChi-square test: Association between Process Variant and Month")
print(f"Chi-square statistic: {chi2:.4f}, p-value: {p_val:.4f}, degrees of freedom: {dof}")
print(f"Conclusion: {'Significant association' if p_val < 0.05 else 'No significant association'}")




--- Advanced Statistical Tests ---

Chi-square test: Association between Process Variant and Month
Chi-square statistic: 246.4547, p-value: 0.0000, degrees of freedom: 55
Conclusion: Significant association


In [14]:
# 10. Process KPIs and Business Impact
# ==================================

print("\n--- Process KPIs and Business Impact ---")

# Calculate key KPIs
total_amount = case_data['case:Amount'].sum()
avg_amount = case_data['case:Amount'].mean()
amount_per_day = total_amount / (case_data['duration_days'].sum())
avg_processing_cost = avg_activities * 15  # Assuming €15 per activity as processing cost
roi = (avg_amount - avg_processing_cost) / avg_processing_cost * 100

print(f"\nProcess KPIs:")
print(f"Total declaration amount processed: €{total_amount:.2f}")
print(f"Average declaration amount: €{avg_amount:.2f}")
print(f"Amount processed per day: €{amount_per_day:.2f}")
print(f"Estimated average processing cost per case: €{avg_processing_cost:.2f}")
print(f"Estimated ROI: {roi:.2f}%")

# Calculate KPIs by variant
variant_kpis = variant_performance.copy()
variant_kpis['processing_cost'] = variant_kpis['activity_count_mean'] * 15
variant_kpis['roi'] = (variant_kpis['case:Amount_mean'] - variant_kpis['processing_cost']) / variant_kpis['processing_cost'] * 100

# Visualize KPIs by variant
fig = px.bar(
    variant_kpis,
    x='variant_id',
    y='roi',
    color='variant_id',
    text='roi',
    title='Estimated ROI by Process Variant',
    labels={'roi': 'ROI (%)', 'variant_id': 'Process Variant'}
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

print("\n--- Analysis Complete ---")


--- Process KPIs and Business Impact ---

Process KPIs:
Total declaration amount processed: €907406.18
Average declaration amount: €86.42
Amount processed per day: €7.50
Estimated average processing cost per case: €80.62
Estimated ROI: 7.19%



--- Analysis Complete ---
